# Replica project

### The data

Stores the final deduplicated datasets and shows how the search can be done with pretrained embeddings

In [1]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd

from torch import nn
import torchvision.models as models
from tqdm import tqdm
import torch
import pickle
from glob import glob
# from scipy import sparse
import sys
import os.path
from os import path


sys.path.insert(0, "../model/")
from utils import *
from model_replica import ReplicaNet


path = '/home/guhennec/scratch/2021_Cini/TopologicalAnalysis_Cini/data/'
data_dir = '/scratch/students/schaerf/'
replica_dir = '/mnt/project_replica/datasets/cini/'
wga_dir = '/mnt/project_replica/datasets/'



In [121]:
metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')
metadata['path'] = replica_dir + metadata['Drawer'].astype(str) + '/' + metadata['Drawer'].astype(str) + '_' + metadata['ImageNumber'].astype(str) + '.jpg'
metadata.head(5)

/tmp/ipykernel_4067631/30539250.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')


,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,AuthorURL,AuthorComplemented,AuthorComplement,...,AuthorBirthLong,AuthorBirthLat,AuthorDeathLong,AuthorDeathLat,AuthorBirthCity,AuthorDeathCity,uid,CardboardURL,ImageURL,path
0,1A,1,ALLEGRINI Francesco,tca con penna acquarellata 148 x 203 mm,"ALLEGRINI, Francesco","Allegrini, Francesco",ulan:500115272,http://vocab.getty.edu/page/ulan/500115272,0,No complement,...,NaN,NaN,NaN,NaN,NaN,NaN,253993c139284a45be233a13121ddeeb,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_1....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,/mnt/project_replica/datasets/cini/1A/1A_1.jpg
1,1A,2,BAROCCI Federico,Studio di figura penna acquarellata (carboncin...,"BAROCCI, Federico","Barocci, Federico",ulan:500115210,http://vocab.getty.edu/page/ulan/500115210,0,No complement,...,12.633333,43.716667,12.633333,43.716667,Urbino,Urbino,1323356994c24635a11fdcd9d5f9284a,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_2....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,/mnt/project_replica/datasets/cini/1A/1A_2.jpg
2,1A,3,BASSANO Leandro,Scene penna su carta blu 171 x 219 mm,"BASSANO, Leandro","Bassano, Leandro",ulan:500015945,http://vocab.getty.edu/page/ulan/500015945,0,No complement,...,11.733333,45.766667,12.326667,45.438611,Bassano del Grappa,Venice,a4268385f6384e61a3dd092bc6b8c083,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_3....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,/mnt/project_replica/datasets/cini/1A/1A_3.jpg
3,1A,4,CAMPIGLI Massimo,Ragazza su una scala gessetto su carta bruna g...,"CAMPIGLI, Massimo","Campigli, Massimo",ulan:500029770,http://vocab.getty.edu/page/ulan/500029770,0,No complement,...,13.400000,52.516667,6.639811,43.269316,Berlin,Var,550f368cdb4442aab4d5e2ada702d6ad,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_4....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,/mnt/project_replica/datasets/cini/1A/1A_4.jpg
4,1A,5,CARRACCI A attr,Madonna e Santi penna 291 x 205 mm,"CARRACCI, A. attr.",NaN,NaN,NaN,1,attr,...,NaN,NaN,NaN,NaN,NaN,NaN,6047de5547b643cb87491be925748bee,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_5....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,/mnt/project_replica/datasets/cini/1A/1A_5.jpg


In [ ]:
metadata.columns

## Full data

In [ ]:
#metadata_2 = pd.read_excel(wga_dir + 'catalog.xls') #no uid
#print(metadata_2)

#metadata_2 = pd.read_csv(wga_dir + 'index.csv',).drop('Unnamed: 0', axis=1)
#metadata_2.columns = ['Author', 'AuthorBirth', 'Description', 'path', 'BeginDate', 'date_num', 'height', 'width', 'is_color', 'uid']

with open(data_dir + 'image_data_2022_04_28.pkl', 'rb') as infile:
    metadata_2 = pickle.load(infile)


metadata_2 = metadata_2[metadata_2['source'] == 'wga']   
metadata_2.head()


,uid,added,source,path,o_uid,o_title,o_date_begin,o_date_end,o_attribution,o_metadata
10789,2355b564-f41e-41a0-9bf6-ae2a503e152a,2017-07-14 15:34:43.288743+00:00,wga,WGA/images/a/aachen/allegory.jpg,0b6ac6d2-934d-4169-8be1-7d1e874be727,Allegory,1598.0,1598.0,Web Gallery of Art,{'URL': 'http://www.wga.hu/html/a/aachen/alleg...
10790,d57ed156-ca10-4cbc-b52d-0ab4f76c84a1,2017-07-14 15:34:43.432937+00:00,wga,WGA/images/a/aachen/bacchus.jpg,c5f370c3-1dcf-41f9-8a18-61e251015e1f,"Bacchus, Ceres and Cupid",1601.0,1650.0,Web Gallery of Art,{'URL': 'http://www.wga.hu/html/a/aachen/bacch...
10791,d51bd5c5-a7b1-4405-87b4-37d428e270a6,2017-07-14 15:34:43.545463+00:00,wga,WGA/images/a/aachen/j_couple.jpg,b1f6412b-2486-48cb-9cc8-dec0d1c373fc,Joking Couple,1601.0,1650.0,Web Gallery of Art,{'URL': 'http://www.wga.hu/html/a/aachen/j_cou...
10792,5d7c0617-643d-4f7e-b58c-f38df03b8eb2,2017-07-14 15:34:43.643147+00:00,wga,WGA/images/a/aachen/rudolf2.jpg,2ec53e3b-d907-495a-964c-304e8b66f1c4,Portrait of Emperor Rudolf II,1590.0,1600.0,Web Gallery of Art,{'URL': 'http://www.wga.hu/html/a/aachen/rudol...
10793,b4de719d-8ca8-4709-a1ff-fafe7bb89db0,2017-07-14 15:34:43.719061+00:00,wga,WGA/images/a/aachen/selfport.jpg,8f58673a-bc0a-45ca-85ff-7b205a8063bf,Self-Portrait with a Glass of Wine,1591.0,1601.0,Web Gallery of Art,{'URL': 'http://www.wga.hu/html/a/aachen/selfp...


In [ ]:
metadata_2.reset_index()['o_metadata'][0]

{'URL': 'http://www.wga.hu/html/a/aachen/allegory.html',
 'DATE': 1598,
 'FORM': 'painting',
 'TYPE': 'mythological',
 'TITLE': 'Allegory',
 'AUTHOR': 'AACHEN, Hans von',
 'SCHOOL': 'German',
 'LOCATION': 'Alte Pinakothek, Munich',
 'TIMELINE': '1601-1650',
 'BORN-DIED': '(b. 1552, Köln, d. 1615, Praha)',
 'TECHNIQUE': 'Oil on copper, 56 x 47 cm'}

In [ ]:
metadata_2['Author'] = metadata_2['o_metadata'].apply(lambda x: x['AUTHOR'])
metadata_2['AuthorBirth'] = metadata_2['o_metadata'].apply(lambda x: x['BORN-DIED'])
metadata_2['Description'] = metadata_2['o_metadata'].apply(lambda x: x['TITLE'])
metadata_2['BeginDate'] = metadata_2['o_metadata'].apply(lambda x: x['DATE'])

metadata_2['uid'] = metadata_2['uid'].astype(str).apply(lambda x: x.replace('-', ''))
metadata_2['path'] = wga_dir + metadata_2['path']

metadata_2 = metadata_2[['Author', 'AuthorBirth', 'Description', 'path', 'BeginDate', 'uid',]]
metadata_2.head()


,Author,AuthorBirth,Description,path,BeginDate,uid
10789,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Allegory,/mnt/project_replica/datasets/WGA/images/a/aac...,1598,2355b564f41e41a09bf6ae2a503e152a
10790,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Bacchus, Ceres and Cupid",/mnt/project_replica/datasets/WGA/images/a/aac...,-,d57ed156ca104cbcb52d0ab4f76c84a1
10791,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Joking Couple,/mnt/project_replica/datasets/WGA/images/a/aac...,-,d51bd5c5a7b1440587b437d428e270a6
10792,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Portrait of Emperor Rudolf II,/mnt/project_replica/datasets/WGA/images/a/aac...,1590s,5d7c0617643d4f7eb58cf38df03b8eb2
10793,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Self-Portrait with a Glass of Wine,/mnt/project_replica/datasets/WGA/images/a/aac...,c. 1596,b4de719d8ca84709a1fffafe7bb89db0


In [ ]:
metadata_2[metadata_2['uid'] == 'fd08dd6652684417a7809bf727111d6f']

,Author,AuthorBirth,Description,path,BeginDate,uid
30114,LEONARDO da Vinci,"(b. 1452, Vinci, d. 1519, Cloux, near Amboise)",Virgin of the Rocks,/mnt/project_replica/datasets/WGA/images/l/leo...,1495-1508,fd08dd6652684417a7809bf727111d6f


In [ ]:
metadata_2[metadata_2['AuthorBirth'] == '(b. 1794, St. Petersburg, d. 1855, St. Petersburg)']

,Author,AuthorBirth,Description,path,BeginDate,uid
50014,"VITALI, Ivan Petrovich","(b. 1794, St. Petersburg, d. 1855, St. Petersb...",Portrait of the Painter Karl Bryullov,/mnt/project_replica/datasets/WGA/images/v/vit...,1852,f997f8df2482498f93360c9511c14b3b
50015,"VITALI, Ivan Petrovich","(b. 1794, St. Petersburg, d. 1855, St. Petersb...",Bust of Dolgoruky,/mnt/project_replica/datasets/WGA/images/v/vit...,1844,74dc4106aa874ed3b38ac72cb31aa82c
50016,"VITALI, Ivan Petrovich","(b. 1794, St. Petersburg, d. 1855, St. Petersb...",Bust of Pushkin,/mnt/project_replica/datasets/WGA/images/v/vit...,1837,a3f0bd614de8434c8d26b3df08d763fc
50017,"VITALI, Ivan Petrovich","(b. 1794, St. Petersburg, d. 1855, St. Petersb...",Venus,/mnt/project_replica/datasets/WGA/images/v/vit...,1852,16612395f449440fb9d8fdb2fa617bc2
50018,"VITALI, Ivan Petrovich","(b. 1794, St. Petersburg, d. 1855, St. Petersb...",Venus,/mnt/project_replica/datasets/WGA/images/v/vit...,1852,69d0d257aadb4c15a0defb28c6adc5f5


In [ ]:
metadata_tot = pd.concat([metadata[['Author', 'AuthorBirth', 'Description', 'path', 'BeginDate', 'uid']], 
                          metadata_2[['Author', 'AuthorBirth', 'Description', 'path', 'BeginDate', 'uid']]], axis=0)
print(metadata_tot.shape)
metadata_tot.head()

(370628, 6)


,Author,AuthorBirth,Description,path,BeginDate,uid
0,ALLEGRINI Francesco,1610.0,tca con penna acquarellata 148 x 203 mm,/mnt/project_replica/datasets/cini/1A/1A_1.jpg,1628.0,253993c139284a45be233a13121ddeeb
1,BAROCCI Federico,1528.0,Studio di figura penna acquarellata (carboncin...,/mnt/project_replica/datasets/cini/1A/1A_2.jpg,1546.0,1323356994c24635a11fdcd9d5f9284a
2,BASSANO Leandro,1557.0,Scene penna su carta blu 171 x 219 mm,/mnt/project_replica/datasets/cini/1A/1A_3.jpg,1575.0,a4268385f6384e61a3dd092bc6b8c083
3,CAMPIGLI Massimo,1895.0,Ragazza su una scala gessetto su carta bruna g...,/mnt/project_replica/datasets/cini/1A/1A_4.jpg,1913.0,550f368cdb4442aab4d5e2ada702d6ad
4,CARRACCI A attr,NaN,Madonna e Santi penna 291 x 205 mm,/mnt/project_replica/datasets/cini/1A/1A_5.jpg,NaN,6047de5547b643cb87491be925748bee


In [ ]:
metadata_tot['uid'].nunique(), metadata_tot.shape

In [ ]:
# dropping duplicates

metadata_tot = metadata_tot.groupby('uid').first().reset_index()

In [ ]:
# in jeanne's code
with open(path + 'save_link_data_2018_08_02.pkl', 'rb') as f:
    morpho_graph_complete = pickle.load(f)

# contains uid	img1	img2	type	annotated
morpho_graph_complete.head()

In [ ]:
morpho_graph_complete[morpho_graph_complete['img1'].str.startswith('fd08dd6652684417a7809bf727111d6f')] # virgin of the rocks

In [133]:
morpho_graph_complete['type'].value_counts() # does positive mean similar?

DUPLICATE        114283
POSITIVE           6293
NON-DUPLICATE        60
PROPOSAL             27
Name: type, dtype: int64

In [134]:
morpho_graph_complete['uid'].nunique(), morpho_graph_complete.shape

(120663, (120663, 5))

In [135]:
positives = get_train_test_split(metadata_tot, morpho_graph_complete)
positives.set.value_counts()

(8521, 12)


train    5152
test     2227
val      1142
Name: set, dtype: int64

In [152]:
positives = positives.groupby('uid_connection').first().reset_index()
print(positives.shape)
positives.head()

(4966, 14)


,uid_connection,index,uid,Author,AuthorBirth,Description,path,BeginDate,img1,img2,type,annotated,cluster,set
0,002275ab17774b55a2d88d7a773b2d93,1289,4e6e928899224c9b8e5d15e37e211fe3,"YÁÑEZ DE LA ALMEDINA, Fernando",(active 1505-36),Madonna and Child with Infant St John,/mnt/project_replica/datasets/WGA/images/y/yan...,c. 1505,a2b4fec1607143d4ad4214b48bab69d6,4e6e928899224c9b8e5d15e37e211fe3,POSITIVE,2016-05-30 13:17:33+00:00,118,train
1,002ca9c06a704c779e69d93cb4fadaa3,1103,45588b4e33444ed8b34c523bac808bc4,TIZIANO Vecellio (copia da),1483.0,S.Mariá Maddalena. cm. 123 x 91,/mnt/project_replica/datasets/cini/158B/158B_5...,c. 1565,45588b4e33444ed8b34c523bac808bc4,da54c106180449d1a52993e9a5bc17fd,POSITIVE,2018-07-24 14:58:44.559049+00:00,50,train
2,0038ab462149452b98a4d7359a06f9f4,2937,ae828b07e4284d80b652206ffb77278a,TIZIANO Vecellio (copia da),1483.0,"Morte di S. Pietro Martire. mm. 123,8 x 84,2",/mnt/project_replica/datasets/cini/158C/158C_1...,None,ae828b07e4284d80b652206ffb77278a,ea64207684694d379579998f88b7c29a,POSITIVE,2018-07-16 09:47:57.987515+00:00,51,val
3,00474897de044f7fb1ef16888dc198f3,2026,76b2c4aeab404bc79768117e3ddc62b1,BELLINI Giovanni (cerchia di),1431.0,Pietà.,/mnt/project_replica/datasets/cini/47A/47A_858...,1449.0,76b2c4aeab404bc79768117e3ddc62b1,b8dd2c0c2c7146e68e5bbb0aaf6cf803,POSITIVE,2018-03-14 10:46:45.603341+00:00,498,test
4,008b01e279594f0eae9419cf52f6262b,2954,b03f0d316b824923a52e79688ba7e1b7,"ALBANI, Francesco","(b. 1578, Bologna, d. 1660, Bologna)",Diana and Actaeon,/mnt/project_replica/datasets/WGA/images/a/alb...,c. 1617,b03f0d316b824923a52e79688ba7e1b7,2264411f35ed4fa18a4962413c13f529,POSITIVE,2016-09-13 15:43:39+00:00,242,train


## Getting a subset of the data

In order to select a subset of the data, we would like to vary spatially and temporally, for it to be suitable to the data in the morphograph and to be comprehensive of what we want to study.


In [137]:
metadata_tot['BeginDate'].fillna(0).astype(str).str.strip('c. ').str[:-2].str.strip('-')

0         1529
1         1600
2         1501
3         1444
4         1623
          ... 
370101        
370102        
370103    1659
370104    1762
370105    1323
Name: BeginDate, Length: 370106, dtype: object

In [138]:
#subset_max = metadata[metadata['BeginDate'] > 1550][metadata['BeginDate'] < 1650][metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()

metadata_tot['BeginDate'] = [int(met) if (len(met) == 4) and ('/' not in met) else 0 for met in metadata_tot['BeginDate'].fillna(0).astype(str).str.strip('c. ').str[:-2].str.strip('-, ')]
metadata_tot.shape

(370106, 6)

In [139]:
subset_max = metadata_tot[metadata_tot['BeginDate'] > 1450][metadata_tot['BeginDate'] < 1650]#[metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()
subset_max.shape

/tmp/ipykernel_4067631/795626849.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset_max = metadata_tot[metadata_tot['BeginDate'] > 1450][metadata_tot['BeginDate'] < 1650]#[metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()


(106617, 6)

In [140]:
subset_max = remove_duplicates(subset_max, morpho_graph_complete)
subset_max.shape

(84950, 13)

In [96]:
list_downloaded = ['/'.join(file.split('/')[-2:]) for file in glob('/mnt/project_replica/datasets/cini/*/*')]
list_downloaded[0]

'74B/74B_22.jpg'

In [97]:
len(list_downloaded)

330682

In [99]:
list_downloaded_2 = ['/'.join(file.split('/')[-2:]) for file in glob(wga_dir + 'WGA/images/*/*/*/*')]
list_downloaded_2[0]

'lambert/child_lo.jpg'

In [100]:
len(list_downloaded_2)

19162

In [14]:
del morpho_graph_complete
del metadata

In [141]:
#uniques = subset_max[subset_max['uid'].isin(list_downloaded)].groupby('uid').first().reset_index()
#subset_max['uid2'] = subset_max['Drawer'].astype(str) + '/' + subset_max['Drawer'].astype(str) + '_' + subset_max['ImageNumber'].astype(str) + '.jpg'

#uniques = subset_max[subset_max['uid2'].isin(list_downloaded)].groupby('uid').first().reset_index()
uniques = subset_max[subset_max['path'].notnull()].groupby('uid').first().reset_index()

print(uniques.shape)

(84950, 13)


In [142]:
subset_max['path'].nunique()

84950

In [16]:
#uniques['Desc'] = uniques['Description'].str.lower().str.split()
#words = uniques.explode('Desc').reset_index()['Desc'].value_counts()
#print(words[words.index.str.len() > 4][words > 100].to_string())

In [143]:
uniques = uniques[uniques['Description'].notnull()]

In [144]:
short = uniques[uniques['Description'].str.split().str.len() == 1]
#print(short['Description'])
short['Desc'] = short['Description'].str.lower().str.strip('.').str.split()
words = short.explode('Desc').reset_index()['Desc'].value_counts()
for idx in words[words.index.str.len() > 4][words > 10].index:
    #print(idx)
    continue

/tmp/ipykernel_4067631/1446786371.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short['Desc'] = short['Description'].str.lower().str.strip('.').str.split()


In [153]:
to_remove = ['affreschi', 'tavolo', 'affresco', 'veduta', 'disegno', 'composizione', 'stucchi', 'studi', ]
short_remove = short.explode('Desc').reset_index()
short_remove = short_remove[short_remove['Desc'].isin(to_remove)]
uniques = uniques[~uniques['uid'].isin(list(short_remove['uid']))]
uniques.shape

(84436, 13)

In [154]:
uniques['uid_connection'] = uniques['uid_connection'].fillna('Not uid connection') 

In [155]:
data = pd.concat([uniques, positives], axis=0).reset_index().groupby(['uid', 'uid_connection']).last().reset_index()
data_sample = pd.concat([uniques.sample(10000), positives], axis=0).reset_index()#.groupby('uid').first().reset_index()
print(data.shape)
#data = data[data['Drawer'].notnull()]
#data_sample = data_sample[data_sample['Drawer'].notnull()]
print(data_sample.shape)
#data['path'] = data['Drawer'].astype(str) + '/' + data['Drawer'].astype(str) + '_' + data['ImageNumber'].astype(str) + '.jpg'
#data_sample['path'] = data_sample['Drawer'].astype(str) + '/' + data_sample['Drawer'].astype(str) + '_' + data_sample['ImageNumber'].astype(str) + '.jpg'

(89402, 15)
(14966, 15)


In [156]:
uid2path = {}

for uid, path in tqdm(zip(data['uid'].unique(), data['path'].unique())):
    uid2path[uid] = path

with open(data_dir + 'uid2path.pkl', 'wb') as outfile:
    pickle.dump(uid2path, outfile)

85773it [00:00, 1155630.06it/s]


In [157]:
data['uid'].nunique(), data['path'].nunique()

(85773, 85773)

In [158]:
data.sample(5)

,uid,uid_connection,level_0,cluster,index,Author,AuthorBirth,Description,path,BeginDate,img1,img2,type,annotated,set
28769,47fe31663b6f4c02b3b7e7218a98bf6a,Not uid connection,27585,47fe31663b6f4c02b3b7e7218a98bf6a,35251,CARRACCI Agostino,1557.0,Tritone di profilo verso destra.,/mnt/project_replica/datasets/cini/1A/1A_196.jpg,1575.0,None,None,None,NaT,None
36366,5d2db105280e4ce796d8fe11456e5cb7,Not uid connection,34731,5d2db105280e4ce796d8fe11456e5cb7,45136,CiMA da CONEGLIANO (e bottega),1455.0,L 'Arcangelo Gabriele. Part.,/mnt/project_replica/datasets/cini/71C/71C_175...,1473.0,None,None,None,NaT,None
49965,8396e31fa5a94162b41a7d1129d203c2,Not uid connection,47544,8396e31fa5a94162b41a7d1129d203c2,63673,BOSCHINI Marco,1605.0,Pag. 668.,/mnt/project_replica/datasets/cini/42C/42C_698...,1623.0,None,None,None,NaT,None
13592,20a95f4e52044455a001fed5983ebd0f,1e7d248e4bc04f268f7253a0e173d1da,13051,20312,15979,LOTTO Lorenzo,1475.0,L' Adultera. 783,/mnt/project_replica/datasets/cini/127B/127B_2...,1493.0,20a95f4e52044455a001fed5983ebd0f,580fd2beae39494a9009bde01af3a36f,DUPLICATE,2018-07-19 17:28:27.168907+00:00,None
77642,d94fee4819054c608f73d551e0ce637d,Not uid connection,73775,d94fee4819054c608f73d551e0ce637d,105260,ASPERTINI Amico,1474.0,"Libro di schizzi. Scena. Penna, colore a corpo...",/mnt/project_replica/datasets/cini/13B/13B_125...,1492.0,None,None,None,NaT,None


In [159]:
data[data['set'].isin(['train', 'test', 'val'])].shape, data_sample[data_sample['set'].isin(['train', 'test', 'val'])].shape


((4966, 15), (4966, 15))

In [160]:
# data_sample.to_csv(data_dir + 'dedup_data_sample_wga.csv')
data.to_csv(data_dir + 'dedup_data_wga.csv')


In [25]:
#data.to_csv(data_dir + 'full_data.csv')
# data_sample.to_csv(data_dir + 'dedup_data_sample.csv')
# data.to_csv(data_dir + 'dedup_data.csv')


In [2]:
data = pd.read_csv(data_dir + 'dedup_data_sample_wga.csv')
print(data['set'].value_counts())
data[data['set'].isin(['train', 'test', 'val'])].shape, #data_sample[data_sample['set'].isin(['train', 'test', 'val'])].shape

train    2984
test     1314
val       668
Name: set, dtype: int64


((4966, 16),)

In [5]:
keep = []

for i, path_ in enumerate(data['path']):
    if os.path.exists(path_):
        keep.append(i)

len(keep), keep[0]

(14948, 0)

In [6]:
data.shape

(14966, 16)

In [8]:
data = data[data.index.isin(keep)]
print(data.shape)
data.to_csv(data_dir + 'dedup_data_sample_wga.csv')

(14948, 16)


In [4]:
data_old = pd.read_csv(data_dir + 'full_data.csv')
data_old[data_old['set'].isin(['train', 'test', 'val'])].shape

(2862, 47)

### Creating embeddings of all the images

In [6]:
model = models.resnet50(pretrained=True)
#print(model)

model = ReplicaNet('resnext-101', 'cuda')

In [29]:
# newmodel = torch.nn.Sequential(
#     *(list(model.children())[:-2]), nn.AdaptiveMaxPool2d((1,1))
# )
#print(newmodel)


In [16]:
it = 10462
print(data['path'][it])
img = preprocess_image_test(data['path'][it])

img.shape

/mnt/project_replica/datasets/WGA/images/t/tiziano/03_1530s/1madonn.jpg


torch.Size([1, 3, 480, 480])

In [19]:
embedding = get_embedding(img, model, device='cuda')
print(embedding)

[0.02288655 0.01400073 0.0181444  ... 0.01918198 0.02341286 0.01433349]


In [35]:
def catch(path, model, device, resolution):
    try:
        return get_embedding(preprocess_image_test(path, resolution=resolution), model, device=device)
    except Exception as e:
        print(e)
        return 0

In [39]:
catch(data['path'][460], model, 'cuda', 320)

output with shape [1, 320, 320] doesn't match the broadcast shape [3, 320, 320]


0

In [ ]:
embeddings = [[uid, catch(path, model, 'cuda', 320)] for uid,path in tqdm(zip(data['uid'].unique(), data['path'].unique()))]

In [32]:
embeddings_new = get_lower_dimension(embeddings)

In [33]:
embeddings[0].shape, embeddings_new[0].shape 

((2048,), (100,))

### Finding most similar embeddings

In [34]:
tree = make_tree_list(embeddings_new)

In [60]:
train_test = data[data['set'].notnull()].drop(columns=['level_0']).reset_index()

Cs = []
for i in tqdm(range(train_test.shape[0])):
    list_theo = list(train_test[train_test['img1'] == train_test['uid'][i]]['img2']) + list(train_test[train_test['img2'] == train_test['uid'][i]]['img1']) + [train_test['uid'][i]]
    list_ = find_most_similar_list(train_test['uid'][i], tree, embeddings_new, list(data['uid'].unique()), list_theo)
    Cs.append(list_)

len(Cs)


100%|██████████| 2862/2862 [00:10<00:00, 286.12it/s]


2862

In [58]:
print(train_test[train_test['img1'].isin(list_downloaded)].shape)
print(train_test[train_test['img2'].isin(list_downloaded)].shape)

(2299, 47)
(2256, 47)


In [61]:
train_test['C'] = Cs

train_test = train_test[train_test['img1'].isin(list_downloaded)]
train_test = train_test[train_test['img2'].isin(list_downloaded)]

final = train_test[['img1', 'img2', 'C', 'set']].explode('C')
final.columns = ['A', 'B', 'C', 'set']
print(final.shape)
final.sample(5)

(31709, 4)


,A,B,C,set
1771,6fcf1af179fa47488d40564e80856a1e,e100fa4b1d1e4abab65256ac310ab543,6bf773f2e04f4c919ccd6f0b76df2b15,train
1539,ef25d7d6549547b59945a61ae6849d3d,272cd39493464a878264554be785e9db,92fc36e4d04743ec97e441342acd29f1,test
63,59fb95590dda4ff58e63dc0df43e70c3,a043a8ccb7994ef68179ad66929143af,756a2003514740448ffb37eec9351177,train
289,383cd750b1cc425089ff3cf3d2c81df1,8b47beb1f1a6438993f3988e880cfa9c,45d519d9ba884c948b2a0abf1a692599,test
581,3d7b289477ca4765a6932bd85cf51fec,246ca4d9a3324ad2aad8ec6a4e1c6769,4ced1a6287c1417db8b41e79e1a04c79,test


In [62]:
final[final['C'].isin(list_downloaded)].shape

(31709, 4)

In [63]:
final[final['set'] == 'train'].reset_index().to_csv(data_dir + 'abc_train.csv')
final[final['set'] == 'test'].reset_index().to_csv(data_dir + 'abc_test.csv')
